In [1]:
!pip install datasets peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 12.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "EMBEDDIA/sloberta"

In [3]:
from datasets import load_dataset

dataset = load_dataset("lenatr99/Slovene_SuperGLUE_RTE")

Generating train split:   0%|          | 0/232 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/29 [00:00<?, ? examples/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/800k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

In [5]:
id2label = {0: "not_entailment", 1: "entailment"}
label2id = {"not_entailment": 0, "entailment": 1}

In [6]:
CONTEXT_COL = "premise"
HYPOTHESIS_COL = "hypothesis"
LABEL_COL = "label"

def preprocess_function(examples):
    context_inputs = [f"{CONTEXT_COL} : {x} " for x in examples[CONTEXT_COL]]
    hypothesis_inputs = [f"{HYPOTHESIS_COL} : {x} " for x in examples[HYPOTHESIS_COL]]
    inputs = [context_input + hypothesis_input for context_input, hypothesis_input in zip(context_inputs, hypothesis_inputs)]

    tokenized_examples = tokenizer(inputs, truncation=True)
    tokenized_examples[LABEL_COL] = [label2id[label] for label in examples[LABEL_COL]]

    return tokenized_examples

In [7]:
tokenized_dataset = dataset.map(preprocess_function, remove_columns=['idx', 'hypothesis', 'premise'], batched=True)

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [8]:
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
from transformers import set_seed

set_seed(42)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(id2label), id2label=id2label, label2id=label2id
)
model.config.use_cache = False

config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/sloberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from peft import LoraConfig, get_peft_model, TaskType

lora_alpha = 32
lora_rank_dropout = 0.1
lora_module_dropout = 0.0
lora_r = 16

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=['query', 'value'],
    base_model_name_or_path=model_name
)

In [12]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,181,954 || all params: 111,805,444 || trainable%: 1.0572


In [13]:
from transformers import TrainingArguments

new_model_name = "lora_fine_tuned_rte_sloberta"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [15]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
50,0.698700,0.697361,0.448276,0.313410
100,0.691800,0.689119,0.586207,0.433283
150,0.694000,0.696396,0.379310,0.276407
200,0.691900,0.697098,0.379310,0.276407
250,0.690400,0.692997,0.448276,0.444335
300,0.693600,0.692338,0.482759,0.482759
350,0.690100,0.691350,0.655172,0.649077
400,0.687400,0.691715,0.551724,0.553898


TrainOutput(global_step=400, training_loss=0.6922356510162353, metrics={'train_runtime': 36.3932, 'train_samples_per_second': 87.928, 'train_steps_per_second': 10.991, 'total_flos': 271997669372352.0, 'train_loss': 0.6922356510162353, 'epoch': 13.793103448275861})

In [16]:
trainer.evaluate()

{'eval_loss': 0.6917153596878052,
 'eval_accuracy': 0.5517241379310345,
 'eval_f1': 0.5538975966562173,
 'eval_runtime': 0.1328,
 'eval_samples_per_second': 218.365,
 'eval_steps_per_second': 30.119,
 'epoch': 13.793103448275861}

In [18]:
trainer.push_to_hub(new_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


events.out.tfevents.1716452961.2e329e5d4728.215.0:   0%|          | 0.00/10.0k [00:00<?, ?B/s]

events.out.tfevents.1716453002.2e329e5d4728.215.1:   0%|          | 0.00/457 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/800k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/4.74M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lenatr99/lora_fine_tuned_rte_sloberta/commit/42fbfe76f630440ab2f8808a650663daea8740fe', commit_message='lora_fine_tuned_rte_sloberta', commit_description='', oid='42fbfe76f630440ab2f8808a650663daea8740fe', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
from transformers import pipeline

set_seed(42)

adapter_name = "lenatr99/" + new_model_name

# Example
text="premise : Potem ko je sklenila zavezništvo z glavno proti-sirijsko koalicijo, so njegovo ženo Strido na nedavnih volitvah izvolili v parlament. hypothesis : Strido so izvolili v parlament."

classifier = pipeline("sentiment-analysis", model=adapter_name)
classifier(text)

adapter_config.json:   0%|          | 0.00/669 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/sloberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_1', 'score': 0.5385059118270874}]